<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Word2vect


In [1]:
import numpy as np

In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [13]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [16]:

def get_vocabulary(corpus):
  corpus_str = ""
  for val in corpus:
    corpus_str = corpus_str + val + " "
  return set(corpus_str.split())

vocabulary = get_vocabulary(corpus)
print(vocabulary)

{'dia', 'hoy', 'de', 'que', 'gracias', 'es', 'el', 'muchas', 'martes'}


### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [35]:
def get_one_hot(corpus, vocabulary):
  one_hot = np.zeros((len(corpus), len(vocabulary)))
  for row, sentence in enumerate(corpus):
    for col, val in enumerate(vocabulary):
      one_hot[row, col] = 1 if val in sentence.split() else 0 
  return one_hot

one_hot = get_one_hot(corpus, vocabulary)
print(one_hot)

[[1. 1. 0. 1. 0. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0. 1. 1. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 1. 1.]]


### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [32]:
def get_frequency(corpus, vocabulary):
  frequency = np.zeros((len(corpus), len(vocabulary)))
  for row, sentence in enumerate(corpus):
    for col, val in enumerate(vocabulary):
      frequency[row, col] = sentence.split().count(val)
  return frequency

frequency = get_frequency(corpus, vocabulary)
print(frequency)

[[1. 1. 0. 1. 0. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0. 1. 1. 0. 2.]
 [0. 0. 0. 0. 1. 0. 0. 1. 1.]]


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [57]:
import math

def get_tfidf(frequency, one_hot, vocabulary):
  idf = np.zeros(len(vocabulary))
  document_freq = np.sum(one_hot, axis=0)
  for i, val in enumerate(vocabulary):
    idf[i] = math.log10(frequency.shape[0]/ document_freq[i])
  tfidf = np.multiply(frequency, idf) 
  return tfidf

tfidf = get_tfidf(frequency, one_hot, vocabulary)
tfidf

array([[0.17609126, 0.17609126, 0.        , 0.47712125, 0.        ,
        0.17609126, 0.        , 0.        , 0.        ],
       [0.17609126, 0.17609126, 0.47712125, 0.        , 0.        ,
        0.17609126, 0.47712125, 0.        , 0.35218252],
       [0.        , 0.        , 0.        , 0.        , 0.47712125,
        0.        , 0.        , 0.47712125, 0.17609126]])

### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [89]:
def get_similarity(corpus, index):
  vocabulary = get_vocabulary(corpus)
  one_hot = get_one_hot(corpus, vocabulary)
  frequency = get_frequency(corpus, vocabulary)
  tfidf = get_tfidf(frequency, one_hot, vocabulary)
  sim_dict = {}
  for i in range(tfidf.shape[0]):
    similarity = cosine_similarity(tfidf[index], tfidf[i])
    #print(f"Document similarity doc {index}, doc {i}: {similarity:.2f}")
    sim_dict[corpus[i]] = similarity
  return sorted(sim_dict.items(), key=lambda item: item[1], reverse=True)

    
get_similarity(corpus, 2)


[('martes muchas gracias', 1.0),
 ('martes el dia de hoy es martes', 0.10845711727883083),
 ('que dia es hoy', 0.0)]